In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../data/checking-logs.sqlite')

In [ ]:
conn.executescript("""
DROP TABLE IF EXISTS datamart;

CREATE TABLE datamart AS
SELECT
    c.uid,
    c.labname,
    c.first_commit_ts,
    p.first_view_ts
FROM
    (
        SELECT
            uid,
            labname,
            MIN(timestamp) AS first_commit_ts
        FROM checker
        WHERE status = 'ready'
          AND numTrials = 1
          AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
          AND uid GLOB 'user_*'
        GROUP BY uid, labname
    ) c
LEFT JOIN
    (
        SELECT
            uid,
            MIN(datetime) AS first_view_ts
        FROM pageviews
        WHERE uid GLOB 'user_*'
        GROUP BY uid
    ) p
ON c.uid = p.uid;
""")

datamart = pd.read_sql(
    "SELECT uid, labname, first_commit_ts, first_view_ts FROM datamart;",
    conn,
    parse_dates=["first_commit_ts", "first_view_ts"]
)


In [ ]:
test = datamart[datamart['first_view_ts'].notna()].copy()
control = datamart[datamart['first_view_ts'].isna()].copy()

mean_first_view_ts = test['first_view_ts'].mean()
control['first_view_ts'] = control['first_view_ts'].fillna(mean_first_view_ts)

In [ ]:
test.to_sql('test', conn, if_exists='replace', index=False)
control.to_sql('control', conn, if_exists='replace', index=False)

In [ ]:
conn.close()

In [ ]:
print(datamart.dtypes)
print(test.shape, control.shape)